In [1]:
import sys
# TO CHANGE
BASEDIR = "../../../../.."
sys.path.insert(0, BASEDIR)

In [35]:
from pprint import pprint
import gc

In [10]:
from src.kg_model.embeddings_model import EmbeddingsModelConfig, EmbeddingsModel
from src.db_drivers.vector_driver import EmbedderModel, EmbedderModelConfig
from src.utils import TripletCreator, NodeCreator, RelationCreator

## Пример работы с EmbeddingsModel-частью модели графа знаний

1. Инициализация класса, выполняющего функции embedder-модели

In [11]:
EMBEDDER_MODEL_PATH = '../../../../../models/intfloat/multilingual-e5-small' # PATH TO APPROPRIATE EMBEDDER-MODEL

In [12]:
embedder_config = EmbedderModelConfig(model_name_or_path=EMBEDDER_MODEL_PATH)
pprint(embedder_config)

EmbedderModelConfig(model_name_or_path='../../../../../models/intfloat/multilingual-e5-small',
                    prompts={'passage': 'passage: ', 'query': 'query: '},
                    device='cuda',
                    normalize_embeddings=True)


In [13]:
embedder = EmbedderModel(embedder_config)

No sentence-transformers model found with name ../../../../../models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


In [22]:
embedders_mapping = {
    'nodes_dense': embedder,
    'triplets_dense': embedder
}

In [14]:
PASSAGE_EXAMPLE1 = "Шла саша по шоссе и сосала сушку."
QUERY_EXAMPLE1 = "Корабли лавировали да не вылавировали."

PASSAGE_EMBEDDING1 = embedder.encode_passages([PASSAGE_EXAMPLE1])
print("passage-emb size:", len(PASSAGE_EMBEDDING1[0]))
QUERY_EMBEDDING1 = embedder.encode_passages([QUERY_EXAMPLE1])
print("query-emb size:", len(QUERY_EMBEDDING1[0]))

passage-emb size: 384
query-emb size: 384


2. Инициализация EmbeddingsModel-класса 

In [15]:
graph_embeddings_config = EmbeddingsModelConfig()
pprint(graph_embeddings_config)

EmbeddingsModelConfig(nodesdb_driver_configs_mapping={'nodes_dense': VectorDriverConfig(db_vendor='chroma',
                                                                                        vector_category='dense',
                                                                                        db_config=VectorDBConnectionConfig(db_info={'db': 'default_db',
                                                                                                                                    'table': 'dense_nodes'},
                                                                                                                           params={'hnsw:M': 4096,
                                                                                                                                   'hnsw:space': 'ip'},
                                                                                                                           need_to_clear=False,
                           

In [23]:
graph_embeddings = EmbeddingsModel(embedders_mapping, graph_embeddings_config)

3. Добавление триплетов в векторную модель

In [24]:
TRIPLET_EXAMPLE1 = TripletCreator.create(
    start_node=NodeCreator.create(n_type='object', name='саша'), 
    relation=RelationCreator.create(r_type='simple', name='шла'), 
    end_node=NodeCreator.create(n_type='object', name='шоссе'))
TRIPLET_EXAMPLE2 = TripletCreator.create(
    start_node=NodeCreator.create(n_type='object', name='саша'), 
    relation=RelationCreator.create(r_type='simple', name='сосала'), 
    end_node=NodeCreator.create(n_type='object', name='сушка'))
TRIPLET_EXAMPLE3 = TripletCreator.create(
    start_node=NodeCreator.create(n_type='object', name='корабли'), 
    relation=RelationCreator.create(r_type='simple', name='выполняли'), 
    end_node=NodeCreator.create(n_type='object', name='лавировать'))
TRIPLET_EXAMPLE4 = TripletCreator.create(
    start_node=NodeCreator.create(n_type='object', name='корабли'), 
    relation=RelationCreator.create(r_type='simple', name='не удалось'), 
    end_node=NodeCreator.create(n_type='object', name='лавировать'))

In [25]:
pprint(TRIPLET_EXAMPLE1, width=200)
pprint(TRIPLET_EXAMPLE2, width=200)
pprint(TRIPLET_EXAMPLE3, width=200)
pprint(TRIPLET_EXAMPLE4, width=200)

Triplet(start_node=Node(name='саша', type=<NodeType.object: 'object'>, prop={}, stringified='саша', id='72adc4fc2f53b1344fb5b56d6ed81b1f'),
        relation=Relation(name='шла', type=<RelationType.simple: 'simple'>, prop={}, id='ba169e8a4d0f434df4d1fe4d1f83a192'),
        end_node=Node(name='шоссе', type=<NodeType.object: 'object'>, prop={}, stringified='шоссе', id='956c2d5e231f655cee06efa6a32969e9'),
        stringified='саша шла шоссе',
        id='04f01caba8301281a741726d625aa7de')
Triplet(start_node=Node(name='саша', type=<NodeType.object: 'object'>, prop={}, stringified='саша', id='72adc4fc2f53b1344fb5b56d6ed81b1f'),
        relation=Relation(name='сосала', type=<RelationType.simple: 'simple'>, prop={}, id='6ee5950581f9c977e4975366fb8f4ed8'),
        end_node=Node(name='сушка', type=<NodeType.object: 'object'>, prop={}, stringified='сушка', id='36a3bb3d92a00133a36cf37c73799161'),
        stringified='саша сосала сушка',
        id='4de1bab8133d757424ec440edbc5aa46')
Triplet(start_

In [30]:
create_info = graph_embeddings.create_triplets([TRIPLET_EXAMPLE1, TRIPLET_EXAMPLE2, TRIPLET_EXAMPLE3, TRIPLET_EXAMPLE4])
print("Идентификаторы добавленных объектов в векторную модель:")
pprint(create_info)
print("Количество объектов в векторной моделе: ")
pprint(graph_embeddings.count_items(detailed=True))

100%|██████████| 1/1 [00:02<00:00,  2.54s/it]

Идентификаторы добавленных объектов в векторную модель:
{'nodes': {'36a3bb3d92a00133a36cf37c73799161',
           '6b6465eaad351e33067e9b2c077591e1',
           '72adc4fc2f53b1344fb5b56d6ed81b1f',
           'ff18960d2ed658400471df1e8fc5dc7c'},
 'triplets': {'6ee5950581f9c977e4975366fb8f4ed8',
              'af8aab01e640a412df17b71e35c02855'}}
Количество объектов в векторной моделе: 
{'nodes': {'episodic': {'nodes_dense': 0, 'nodes_sparse_bm25': 0},
           'hyper': {'nodes_dense': 0, 'nodes_sparse_bm25': 0},
           'object': {'nodes_dense': 5, 'nodes_sparse_bm25': 5}},
 'triplets': {'triplets_dense': 4}}


In [31]:
graph_embeddings.check_consistency()

4. Удаление триплетов из векторной модели

In [33]:
graph_embeddings.delete_triplets([TRIPLET_EXAMPLE2, TRIPLET_EXAMPLE4])
print("Количество объектов в векторной моделе: ")
pprint(graph_embeddings.count_items(detailed=True))

Количество объектов в векторной моделе: 
{'nodes': {'episodic': {'nodes_dense': 0, 'nodes_sparse_bm25': 0},
           'hyper': {'nodes_dense': 0, 'nodes_sparse_bm25': 0},
           'object': {'nodes_dense': 1, 'nodes_sparse_bm25': 1}},
 'triplets': {'triplets_dense': 2}}


In [29]:
graph_embeddings.check_consistency()

5. Полное удаление содержимого векторной модели

In [34]:
graph_embeddings.clear()
print("Количество объектов в векторной моделе: ")
pprint(graph_embeddings.count_items(detailed=True))

Количество объектов в векторной моделе: 
{'nodes': {'episodic': {'nodes_dense': 0, 'nodes_sparse_bm25': 0},
           'hyper': {'nodes_dense': 0, 'nodes_sparse_bm25': 0},
           'object': {'nodes_dense': 0, 'nodes_sparse_bm25': 0}},
 'triplets': {'triplets_dense': 0}}


In [ ]:
del graph_embeddings
gc.collect()